In [78]:
!pip install --upgrade pip

In [79]:
!pip install tensorflow-datasets

In [82]:
import numpy as np
import tensorflow as tf

In [84]:
import subprocess
import sys

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

install("tensorflow-datasets")

import tensorflow_datasets as tfds


[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: /opt/anaconda3/bin/python -m pip install --upgrade pip


In [86]:
import tensorflow_datasets as tfds

In [88]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

In [90]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_validation_samples, tf.int64)

def scale(image,label):
    image = tf.cast(image, tf.float32)
    image/=255.0
    return image,label

scaled_train_and_validation_data=mnist_train.map(scale)
test_data = mnist_test.map(scale)

In [92]:
BUFFER_SIZE=10000
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

In [94]:
validation_data=shuffled_train_and_validation_data.take(num_validation_samples)
train_data=shuffled_train_and_validation_data.skip(num_validation_samples)

In [96]:
BATCH_SIZE=100
train_data=train_data.batch(BATCH_SIZE)
validation_data=validation_data.batch(num_validation_samples)
test_data=test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

2024-07-15 14:06:21.426568: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [104]:
input_size=784
output_size=10
hidden_layer_size=100

model=tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax'),
        ])

In [106]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [108]:
NUM_EPOCHS = 5
model.fit(train_data, 
          epochs=NUM_EPOCHS, 
          validation_data=(validation_inputs, validation_targets), 
          verbose=2)

Epoch 1/5
540/540 - 1s - 2ms/step - accuracy: 0.9052 - loss: 0.3380 - val_accuracy: 0.9540 - val_loss: 0.1668
Epoch 2/5
540/540 - 1s - 1ms/step - accuracy: 0.9603 - loss: 0.1351 - val_accuracy: 0.9672 - val_loss: 0.1107
Epoch 3/5
540/540 - 1s - 1ms/step - accuracy: 0.9725 - loss: 0.0915 - val_accuracy: 0.9732 - val_loss: 0.0921
Epoch 4/5
540/540 - 1s - 1ms/step - accuracy: 0.9774 - loss: 0.0742 - val_accuracy: 0.9762 - val_loss: 0.0778
Epoch 5/5
540/540 - 1s - 1ms/step - accuracy: 0.9820 - loss: 0.0573 - val_accuracy: 0.9805 - val_loss: 0.0653


In [110]:
test_loss, test_accuracy = model.evaluate(test_data)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.9750 - loss: 0.0799 


In [114]:
print("Test loss : {0:.2f}. Test accuracy: {1:.2f}%".format(test_loss, test_accuracy*100.))

Test loss : 0.08. Test accuracy: 97.51%
